# GIS 5571: `Lab 1`

One way that lab 1 could be accomplished, focused on making use of the `pandas` and `arcgis` packages.

In [1]:
# Imports
import arcgis                   # this enables us to use the ArcGIS API for Python
from io import StringIO         # this enables us to read CSV files directly into a DF (without saving locally)
import pandas as pd             # this enables us to use Pandas DataFrames
import requests                 # this enables us to make API requests/calls


# Optional: For ignoring SSL warnings
import warnings


# Optional: For using Folium to visualize
import folium


# Optional: For using Kepler.GL to visualize
import json

import geopandas as gpd
from keplergl import KeplerGl

## Google Maps

In [2]:
# TODO: REPLACE API KEY
google_maps_key = "PLACE YOUR KEY HERE"

In [3]:
def searchGoogle(lat, long, radius, place_type, keyword, api_key):
    """
    Function to perform a nearby search in Google Places API.
    
    See docs here: https://developers.google.com/maps/documentation/places/web-service/search-nearby

    Args:
        lat (float): Latitude of search
        long (float): Longitude of search
        radius (float): Distance to search (in meters) [optional for API]
        place_type (str): The type of place [optional for API]
        keyword (str): The search terms
        api_key (str): Your API key
        
    Raises:
        Exception: Raised when invalid response is recieved

    Returns:
        dict: A JSON representation of the data (which is really just a Python dictionary with nested lists/dictionaries)
    """
    
    # Base URL for the API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Define your parameters as a dictionary
    parameters = {
        "location": f"{lat},{long}",
        "radius": str(radius),
        "type": place_type,
        "keyword": keyword,
        "key": api_key
    }

    # Make request
    response = requests.get(base_url, params=parameters)

    # Checking response to see if request worked
    if response.status_code == 200:
        data = response.json()

    # If it didn't work, raise an error
    else:
        raise Exception(f"Error: {response.status_code}")

    # Returning the results
    return data["results"]


def convertGoogleToSEDF(data):
    """
    Function that converts a Google Places JSON to an ArcGIS SEDF.

    Args:
        data (dict): A JSON representation of the data

    Returns:
        DataFrame: An ArcGIS SEDF
    """
    # Load your data into a Pandas DataFrame
    df = pd.DataFrame(data)
    
    # Extracting Coordinates
    df["LATITUDE"] = df["geometry"].apply(lambda x: x["location"]["lat"])
    df["LONGITUDE"] = df["geometry"].apply(lambda x: x["location"]["lng"])
    
    # Convert DF to an ArcGIS Spatially-Enabled DataFrame (SEDF)
    sedf = pd.DataFrame.spatial.from_xy(df=df, x_column="LONGITUDE", y_column="LATITUDE", sr=4326)
    
    # Return result
    return sedf

In [4]:
# Get Data
google_json = searchGoogle(44.971635, -93.270123, "100000", "", "charging station", google_maps_key)

google_json[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 44.9738407, 'lng': -93.2301744},
  'viewport': {'northeast': {'lat': 44.97521197989272,
    'lng': -93.22880967010728},
   'southwest': {'lat': 44.97251232010727, 'lng': -93.23150932989272}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
 'icon_background_color': '#7B9EB0',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
 'name': 'Electric Vehicle Charging Station',
 'opening_hours': {'open_now': True},
 'place_id': 'ChIJPYtCHBgts1IRVegTEu_xl3Q',
 'plus_code': {'compound_code': 'XQF9+GW Minneapolis, Minnesota',
  'global_code': '86P8XQF9+GW'},
 'rating': 4.3,
 'reference': 'ChIJPYtCHBgts1IRVegTEu_xl3Q',
 'scope': 'GOOGLE',
 'types': ['point_of_interest', 'establishment'],
 'user_ratings_total': 4,
 'vicinity': '272 SE Harvard St, Minneapolis'}

In [5]:
# Converting to SEDF
google_sedf = convertGoogleToSEDF(google_json)

google_sedf.head()

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,photos,LATITUDE,LONGITUDE,SHAPE
0,OPERATIONAL,"{'location': {'lat': 44.9738407, 'lng': -93.23...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Electric Vehicle Charging Station,{'open_now': True},ChIJPYtCHBgts1IRVegTEu_xl3Q,"{'compound_code': 'XQF9+GW Minneapolis, Minnes...",4.3,ChIJPYtCHBgts1IRVegTEu_xl3Q,GOOGLE,"[point_of_interest, establishment]",4,"272 SE Harvard St, Minneapolis",NaN,44.973841,-93.230174,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,OPERATIONAL,"{'location': {'lat': 45.00449769999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Tesla Supercharger,{'open_now': True},ChIJvyot9T8ts1IRIVq3yLjXijs,"{'compound_code': '2Q3C+QG St Anthony, Minneso...",4.0,ChIJvyot9T8ts1IRIVq3yLjXijs,GOOGLE,"[point_of_interest, establishment]",2,"New Brighton Blvd, St Anthony","[{'height': 3468, 'html_attributions': ['<a hr...",45.004498,-93.228660,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,OPERATIONAL,"{'location': {'lat': 44.9380114, 'lng': -93.28...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Shell Recharge Charging Station,NaN,ChIJj-3rRJcn9ocRpBRQLHjMvnw,"{'compound_code': 'WPQ6+6V Minneapolis, Minnes...",0.0,ChIJj-3rRJcn9ocRpBRQLHjMvnw,GOOGLE,"[point_of_interest, establishment]",0,"Lyndale Ave S, Minneapolis",NaN,44.938011,-93.287821,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,OPERATIONAL,"{'location': {'lat': 44.9715706, 'lng': -93.22...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,ZEF Charging Station,NaN,ChIJyyvRWiIts1IRa6LV3qcLAz4,"{'compound_code': 'XQCF+J5 Minneapolis, Minnes...",0.0,ChIJyyvRWiIts1IRa6LV3qcLAz4,GOOGLE,"[point_of_interest, establishment]",0,"SE Oak St, Minneapolis",NaN,44.971571,-93.227000,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,OPERATIONAL,"{'location': {'lat': 44.98491079999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,ZEF Charging Station,NaN,ChIJVzB5t3Mts1IRw0cLQLsfys8,"{'compound_code': 'XQM3+XV Minneapolis, Minnes...",5.0,ChIJVzB5t3Mts1IRw0cLQLsfys8,GOOGLE,"[point_of_interest, establishment]",1,"6th Ave SE, Minneapolis",NaN,44.984911,-93.245303,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."


## NDAWN

In [6]:
def searchNDAWN(station_id, begin_date, end_date):
    """
    Function to retrieve CSV data from NDAWN for dates within 2023.

    Args:
        station_id (int): A station ID number
        begin_date (str): Start date that you want data for
        end_date (str): End date that you want data for

    Raises:
        Exception: Raised when invalid response is recieved

    Returns:
        str: CSV-formatted string
    """
    # Base URL for the API
    base_url = "https://ndawn.ndsu.nodak.edu/table.csv"
    
    # Define your parameters as a dictionary
    parameters = {
        "station": str(station_id),
        "variable": [
            "ddmxt", "ddmxtt", "ddmnt", "ddmntt", "ddavt", "dddtr", "ddbst", "ddtst",
            "ddws", "ddmxws", "ddmxwst", "ddwd", "ddwdsd", "ddsr", "ddtpetp",
            "ddtpetjh", "ddr", "dddp", "ddwc", "ddmnwc", "ddmxt9", "ddmxtt9",
            "ddmnt9", "ddmntt9", "ddmxws10", "ddmxwst10", "ddwd10", "ddwdsd10"
        ],
        "year": "2023",
        "ttype": "daily",
        "begin_date": begin_date,
        "end_date": end_date
    }
    
    # Make request
    response = requests.get(base_url, params=parameters)

    # Checking response to see if request worked
    if response.status_code == 200:
        data = response.text

    # If it didn't work, raise an error
    else:
        raise Exception(f"Error: {response.status_code}")

    # Returning the results
    return data


def convertNDAWNToSEDF(data):
    """
    Function that converts an NDAWN CSV to an ArcGIS SEDF.

    Args:
        data (dict): A JSON representation of the data

    Returns:
        DataFrame: An ArcGIS SEDF
    """
    # Load your data into a Pandas DataFrame
    df = pd.read_csv(StringIO(data), header=3, skiprows=[4])

    # Convert DF to an ArcGIS Spatially-Enabled DataFrame (SEDF)
    sedf = pd.DataFrame.spatial.from_xy(df=df, x_column="Longitude", y_column="Latitude", sr=4326)
    
    # Return result
    return sedf

In [7]:
# Get Data
ndawn_csv = searchNDAWN(95, "2023-09-15", "2023-09-30")

# Converting to SEDF
ndawn_sedf = convertNDAWNToSEDF(ndawn_csv)

ndawn_sedf.head()

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Max Temp Time,...,Min Temp Time at 9 m Flag,Max Wind Speed at 10 m,Max Wind Speed at 10 m Flag,Max Wind Speed Time at 10 m,Max Wind Speed Time at 10 m Flag,Avg Wind Dir at 10 m,Avg Wind Dir at 10 m Flag,Avg Wind Dir SD at 10 m,Avg Wind Dir SD at 10 m Flag,SHAPE
0,Williams,48.858419,-94.980798,1093,2023,9,15,61.286,NaN,11:10:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."
1,Williams,48.858419,-94.980798,1093,2023,9,16,62.528,NaN,13:24:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."
2,Williams,48.858419,-94.980798,1093,2023,9,17,67.298,NaN,15:48:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."
3,Williams,48.858419,-94.980798,1093,2023,9,18,74.894,NaN,14:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."
4,Williams,48.858419,-94.980798,1093,2023,9,19,76.604,NaN,14:13:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -94...."


## MN Geospatial Commons

In [8]:
def searchGeocommons(search_term):
    """
    Function to search through the MN Geospatial Commons datasets through the CKAN API

    Args:
        search_term (str): Search terms that you want to use

    Raises:
        Exception: Raised when invalid response is recieved

    Returns:
        dict: Dictionary of datasets and their resources
    """
    # Base URL for the API
    base_url = "https://gisdata.mn.gov/api/3/action/package_search"
    
    # Define your parameters as a dictionary
    parameters = {
        "q": search_term
    }
    
    # Ignoring warnings
    warnings.filterwarnings("ignore") 

    # Make request (verify=False is what fixes the SSL error)
    response = requests.get(base_url, params=parameters, verify=False)

    # Checking response to see if request worked
    if response.status_code == 200:
        data = response.json()

    # If it didn't work, raise an error
    else:
        raise Exception(f"Error: {response.status_code}")
    
    # Loop through first five results and get some basic data about them
    summary = {}
    
    for i in range(5):
        result = data["result"]["results"][i]
        summary[result["name"]] = [{j["name"]: j["url"]} for j in result["resources"]]

    # Returning the results
    return summary


def agsRequest(ags_url, layer_number):
    """
    Function to request data from an AGS REST API Feature Service

    Args:
        ags_url (str): URL for the feature service
        layer_number (int): The number of the feature service layer that you want to access data for

    Raises:
        Exception: Raised when invalid response is recieved

    Returns:
        dict: A JSON representation of the data
    """
    # Base URL for the API
    base_url = ags_url + f"/{layer_number}/query"
    
    # Define your parameters as a dictionary
    parameters = {
        "where": "1=1",
        "outFields": "*",
        "outSR": "4326",
        "returnGeometry": "true",
        "f": "pjson"
    }
    
    # Make request
    response = requests.get(base_url, params=parameters)

    # Checking response to see if request worked
    if response.status_code == 200:
        data = response.json()

    # If it didn't work, raise an error
    else:
        raise Exception(f"Error: {response.status_code}")
    
    # Returning the results
    return data


def convertAGSToSEDF(data):
    """
    Function to convert a JSON to a Feature Set and then a SEDF

    Args:
        data (dict): A JSON representation of the data

    Returns:
        DataFrame: An ArcGIS SEDF
    """
    # Load your data into a Feature Set from a JSON (really a dict)
    feature_set = arcgis.features.FeatureSet.from_dict(data)
    
    # Return the SEDF of the feature set
    return feature_set.sdf


In [9]:
# Search Geocommons for datasets
geocommons_search = searchGeocommons("Counties and Cities & Townships, Twin Cities Metropolitan Area")

geocommons_search

{'us-mn-state-metc-bdry-census2010counties-ctus': [{'OGC GeoPackage': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/gpkg_bdry_census2010counties_ctus.zip'},
  {'Esri ArcGIS Server Map Service': 'http://gis2.metc.state.mn.us/arcgis/rest/services/MetroGIS/Demographics/MapServer'},
  {'Shapefile': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/shp_bdry_census2010counties_ctus.zip'},
  {'ESRI File Geodatabase': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/fgdb_bdry_census2010counties_ctus.zip'},
  {'Full Metadata Record': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2010counties_ctus/metadata/metadata.html'}],
 'us-mn-state-metc-bdry-census2020counties-ctus': [{'ESRI File Geodatabase': 'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/bdry_census2020counties_ctus/fgdb_bdry_census2020

In [10]:
# Selecting layer URL to use
geocommons_layer_url = geocommons_search['us-mn-state-metc-bdry-metro-counties-and-ctus'][5]['Esri ArcGIS Server Map Service - CTUs']

# Get data
geocommons_json = agsRequest(geocommons_layer_url, 0)

# Converting to SEDF
geocommons_sedf = convertAGSToSEDF(geocommons_json)

geocommons_sedf.head()

,OBJECTID,CTU_ID,CTU_NAME,CTU_ID_CEN,CTU_CODE,CTU_TYPE,ABC_SORT,FIVE_COLOR,GlobalID,Shape__Area,Shape__Length,SHAPE
0,1,2393887,Afton,02393887,00316,C,2,5,{89697DCB-D7FF-4D9E-9F58-AF87CEF52BA4},68196822.7703,38529.272792,"{""rings"": [[[-92.78148298874046, 44.9489226380..."
1,2,2393954,Andover,02393954,01486,C,4,1,{AA72028F-40FB-48C3-860A-AE887D1739EE},90209577.531665,46098.461917,"{""rings"": [[[-93.32591932922234, 45.2980611081..."
2,3,2393964,Anoka,02393964,01720,C,6,4,{0B34C43B-6E7A-4C0F-8E7D-73278D55B1F9},18429196.152523,24026.015964,"{""rings"": [[[-93.3886170420429, 45.23971403111..."
3,4,2393967,Apple Valley,02393967,01900,C,8,2,{1859687E-512D-4885-A03A-20ECB9941C23},45247136.862762,27901.691112,"{""rings"": [[[-93.15641150315399, 44.7757362011..."
4,5,2393979,Arden Hills,02393979,02026,C,10,1,{420AD298-34FD-4095-B572-25AB53C99912},24847198.256682,23295.79096,"{""rings"": [[[-93.14696225118011, 45.0357718948..."


## Joining

In [11]:
# Check the SR for Charging Station Points (from Google API)
google_sedf.spatial.sr

{'wkid': 4326}

In [12]:
# Check the SR for Twin Cities City Polygons (from Geocommons/Met Council)
geocommons_sedf.spatial.sr

{'wkid': 4326, 'latestWkid': 4326}

In [13]:
# Spatial Join
joined = geocommons_sedf.spatial.join(google_sedf)

joined.head()

,OBJECTID,CTU_ID,CTU_NAME,CTU_ID_CEN,CTU_CODE,CTU_TYPE,ABC_SORT,FIVE_COLOR,GlobalID,Shape__Area,...,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,photos,LATITUDE,LONGITUDE
0,119,2396511,St. Paul,02396511,58000,C,322,3,{3DFDD6FC-CF89-4D78-AB55-A92282B27967},144986744.519118,...,"{'compound_code': 'XRJ4+C5 St Paul, Minnesota'...",0.0,ChIJQ86cCpEss1IRDuZX267DHs8,GOOGLE,"[point_of_interest, establishment]",0,"Como Ave, St Paul",NaN,44.981054,-93.194503
1,142,2395345,Minneapolis,02395345,43000,C,230,1,{292D6DF5-E834-468F-AC28-07C3BF10AC37},148630870.819427,...,"{'compound_code': 'XQF9+GW Minneapolis, Minnes...",4.3,ChIJPYtCHBgts1IRVegTEu_xl3Q,GOOGLE,"[point_of_interest, establishment]",4,"272 SE Harvard St, Minneapolis",NaN,44.973841,-93.230174
2,142,2395345,Minneapolis,02395345,43000,C,230,1,{292D6DF5-E834-468F-AC28-07C3BF10AC37},148630870.819427,...,"{'compound_code': '2Q3C+QG St Anthony, Minneso...",4.0,ChIJvyot9T8ts1IRIVq3yLjXijs,GOOGLE,"[point_of_interest, establishment]",2,"New Brighton Blvd, St Anthony","[{'height': 3468, 'html_attributions': ['<a hr...",45.004498,-93.228660
3,142,2395345,Minneapolis,02395345,43000,C,230,1,{292D6DF5-E834-468F-AC28-07C3BF10AC37},148630870.819427,...,"{'compound_code': 'WPQ6+6V Minneapolis, Minnes...",0.0,ChIJj-3rRJcn9ocRpBRQLHjMvnw,GOOGLE,"[point_of_interest, establishment]",0,"Lyndale Ave S, Minneapolis",NaN,44.938011,-93.287821
4,142,2395345,Minneapolis,02395345,43000,C,230,1,{292D6DF5-E834-468F-AC28-07C3BF10AC37},148630870.819427,...,"{'compound_code': 'XQCF+J5 Minneapolis, Minnes...",0.0,ChIJyyvRWiIts1IRa6LV3qcLAz4,GOOGLE,"[point_of_interest, establishment]",0,"SE Oak St, Minneapolis",NaN,44.971571,-93.227000


In [14]:
# Aggregating data
joined_aggregated = joined.groupby('CTU_NAME', as_index=False).SHAPE.agg({"SHAPE": "first", "station_count": "count"})

joined_aggregated

,CTU_NAME,SHAPE,station_count
0,Minneapolis,"{""rings"": [[[-93.28897034514618, 45.0511361170...",19
1,St. Paul,"{""rings"": [[[-93.13118451796046, 44.9918531162...",1


## Mapping

### Folium Code

In [15]:
def mapGoogle(lat, long, zoom, sedf):
    # Create an empty base map in Folium
    m = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Create list of point gemoetries from GDF
    point_list = [[row["LATITUDE"], row["LONGITUDE"]] for _, row in sedf.iterrows()]
    
    # Loop though point list, add markers to map (w/ Google symbology), and returning map
    i = 0
    
    for feature in point_list:
        custom_icon = folium.features.CustomIcon(sedf.icon[i], icon_size=(14, 14))
        
        m.add_child(folium.Marker(location = feature,
                            popup =
                            "Name: " + str(sedf.name[i]) + '\n' +
                            "Address: " + str(sedf.vicinity[i]) + '\n' +
                            "Rating: " + str(sedf.rating[i]) + '\n' +
                            "Coordinates: " + str(point_list[i]),
                            icon = custom_icon))
        i += 1
        
    return m

In [16]:
def mapNDAWN(lat, long, zoom, sedf):
    # Create an empty base map in Folium
    m = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Create list of point gemoetries from GDF
    point_list = [[row["Latitude"], row["Longitude"]] for _, row in sedf.iterrows()]
    
    # Loop though point list, add markers to map (w/ Google symbology), and returning map
    i = 0
    
    for feature in point_list:
        m.add_child(folium.Marker(location = feature,
                            popup =
                            "Month: " + str(sedf.Month[i]) + '\n' +
                            "Day: " + str(sedf.Day[i]) + '\n' +
                            "Year: " + str(sedf.Year[i]) + '\n' +
                            "Max Temp: " + str(sedf["Max Temp"][i]) + '\n' +
                            "Coordinates: " + str(point_list[i])
                            ))
        i += 1
        
    return m

In [17]:
def mapGeocommons(lat, long, zoom, sedf):
    # Create an empty base map in Folium
    m = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Limiting # Features
    sedf = sedf.iloc[:5]
    
    # Convert to Feature Set
    fs = arcgis.features.FeatureSet.from_dataframe(sedf)
    
    # Add to Map
    folium.GeoJson(fs.to_geojson,
        style_function = lambda x: {"fillOpacity": 0.2, 'fillColor': 'blue'},
        tooltip=folium.features.GeoJsonTooltip(
            fields=['CTU_NAME'],
            labels=False
        )
    ).add_to(m)

    return m

In [18]:
def mapJoined(lat, long, zoom, sedf):
    # Create an empty base map in Folium
    m = folium.Map(location=[lat, long], tiles="Cartodb Positron", zoom_start=zoom)
    
    # Convert to Feature Set
    fs = arcgis.features.FeatureSet.from_dataframe(sedf)
    
    # Add to Map
    folium.GeoJson(fs.to_geojson,
        style_function = lambda x: {"fillOpacity": 0.2, 'fillColor': 'green' if x['properties']['station_count'] > 5 else 'red'},
        tooltip=folium.features.GeoJsonTooltip(
            fields=['CTU_NAME', 'station_count'],
            labels=False
        )
    ).add_to(m)

    return m

### Folium Maps

In [19]:
# Displaying Folium map of Google data
mapGoogle(44.971635, -93.270123, 12, google_sedf)

In [20]:
# Displaying Folium map of NDAWN data (multiple points on top of each other)
mapNDAWN(48.9, -94.95, 10, ndawn_sedf)

In [21]:
# Displaying Folium map of MN Geospatial Commons data (only 5 features - it's big)
mapGeocommons(44.95, -93.2, 10, geocommons_sedf)

In [22]:
# Displaying Folium map of joined data
mapJoined(44.95, -93.2, 11, joined_aggregated)

### Kepler.GL Code

In [23]:
# Helper function for converting SEDF to GDF
def sedfToGdf(sedf):
    if 'geometry' in sedf.columns:
        sedf = sedf.drop(['geometry'], axis=1)
    
    fs = arcgis.features.FeatureSet.from_dataframe(sedf)

    gjs = json.loads(fs.to_geojson)

    gdf = gpd.GeoDataFrame.from_features(gjs)
    
    return gdf

In [24]:
# Create Map
kgl = KeplerGl(height=500)

# Add Data
kgl.add_data(data=sedfToGdf(google_sedf), name='Charging Stations (Google)')
kgl.add_data(data=sedfToGdf(ndawn_sedf), name='NDAWN')
kgl.add_data(data=sedfToGdf(joined_aggregated), name='Charging Stations per City')

# Save as HTML file
kgl.save_to_html(file_name='lab1_kepler.html')

# Display Map
kgl

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to lab1_kepler.html!


KeplerGl(data={'Charging Stations (Google)': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, …